In [1]:
import torch
import torch.nn as nn
import os
import cv2
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [12]:
from torchvision import models
from torchvision import transforms
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix

In [3]:
if(os.path.split(os.getcwd())[1] == "Transfer_Learning"):
    os.chdir("..")
print("Current Working Directory: {}".format(os.path.split(os.getcwd())[1]))

cuda = True

Current Working Directory: progettoVIPM


In [4]:
from utils.loadersAndEnums import datasets
from utils.loadersAndEnums import networks
from utils.loadersAndEnums import ImageDataset

In [5]:
model = models.resnet50()
num_classes = 101
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.load_state_dict(torch.load("C:\\Users\\marco\\Desktop\\Uni\\Magistrale\\VIPM\\Progetto\\resnet50model.pth")['model_state'])
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)

cuda


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
num_classes = 251
model.fc = nn.Linear(model.fc.in_features, num_classes)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [8]:
from torch.utils.data import DataLoader
trainData = datasets.TRAINING_LABELED
testData = datasets.TEST
train = ImageDataset(trainData,224,cuda=True)
test = ImageDataset(testData,224,cuda=True)
trainDL = DataLoader(train, batch_size=128, shuffle=True)
testDL = DataLoader(test, batch_size=128, shuffle=True)


In [9]:
torch.cuda.empty_cache()
batch = next(iter(trainDL))
print(batch)

[tensor([[[[ 8.8068e-01,  8.7828e-01,  8.7597e-01,  ...,  8.6275e-01,
            8.6275e-01,  8.6275e-01],
          [ 8.7403e-01,  8.7173e-01,  8.6934e-01,  ...,  8.6211e-01,
            8.6280e-01,  8.6275e-01],
          [ 8.7405e-01,  8.7156e-01,  8.6933e-01,  ...,  8.6012e-01,
            8.6246e-01,  8.6275e-01],
          ...,
          [ 8.1575e-01,  8.1055e-01,  8.0680e-01,  ...,  7.6588e-01,
            7.7134e-01,  7.8273e-01],
          [ 8.2323e-01,  8.2116e-01,  8.1909e-01,  ...,  8.0416e-01,
            8.0609e-01,  8.1300e-01],
          [ 8.3163e-01,  8.2929e-01,  8.2718e-01,  ...,  8.4629e-01,
            8.4789e-01,  8.5082e-01]],

         [[ 8.8068e-01,  8.7828e-01,  8.7597e-01,  ...,  8.5882e-01,
            8.5882e-01,  8.5882e-01],
          [ 8.7403e-01,  8.7173e-01,  8.6934e-01,  ...,  8.5819e-01,
            8.5887e-01,  8.5882e-01],
          [ 8.7405e-01,  8.7156e-01,  8.6933e-01,  ...,  8.5620e-01,
            8.5854e-01,  8.5883e-01],
          ...,
    

In [10]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in trainDL:
        print(labels)
        labels = [int(label) for label in labels]
        inputs = inputs.to(device) 
        labels = torch.tensor(labels, dtype=torch.long).to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(trainDL)}')

('166', '138', '121', '247', '156', '23', '209', '53', '198', '52', '149', '118', '151', '194', '230', '147', '193', '58', '87', '28', '223', '90', '140', '81', '0', '144', '176', '71', '1', '244', '158', '150', '51', '210', '210', '223', '204', '58', '157', '0', '207', '208', '100', '246', '146', '220', '236', '80', '39', '129', '107', '25', '169', '93', '228', '108', '232', '29', '101', '24', '120', '60', '228', '119', '176', '247', '158', '60', '209', '1', '2', '238', '42', '232', '201', '248', '55', '0', '115', '134', '192', '186', '60', '231', '47', '46', '69', '147', '194', '9', '222', '184', '6', '131', '53', '120', '183', '63', '152', '45', '40', '182', '134', '137', '100', '30', '95', '230', '99', '234', '111', '217', '213', '128', '138', '45', '38', '69', '57', '150', '208', '236', '43', '170', '93', '231', '195', '19')
('48', '31', '201', '74', '130', '130', '15', '14', '205', '183', '163', '43', '123', '2', '39', '21', '75', '56', '234', '92', '16', '153', '192', '172', '19

In [11]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in testDL:
        labels = [int(label) for label in labels]
        inputs, labels = inputs.to(device), torch.tensor(labels, dtype=torch.long).to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total}%')

Accuracy: 40.8370852092713%


In [13]:
all_preds = []
all_labels = []
with torch.no_grad():
    for inputs,labels in testDL:
        labels = [int(label) for label in labels]
        inputs, labels = inputs.to(device), torch.tensor(labels, dtype=torch.long).to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(predicted.cpu().numpy())
        
all_preds = np.array(all_preds)
all_labels = np.array(all_labels)
cm = confusion_matrix(all_labels, all_preds)

np.save('cmResNet50PreTrained10Epochs.npy',cm)
torch.save(model, "finetunedResNet50.pth")
